In [1]:
#Author: Tung Tran
#Student ID: x2020fjw
#Student number: 202005431
#Email: x2020fjw@stfx.ca
import numpy as np
import pandas as pd 
import sklearn as sk 
import matplotlib.pyplot as plt

from collections import Counter

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
feamat = pd.read_csv('feamat.csv')

train[['chemId','assayId']] = train.Id.str.split(';',1,expand=True)
test[['chemId','assayId']] = test.x.str.split(';',1,expand=True)

test_id = test.x
train = train.drop(['Id'],1)
test = test.drop(['x'],1)

feamat = feamat.drop('V2',1)

In [3]:
#Merge feamat with train and test

def merge_feamat(original, feamat):
    original = pd.merge(original,feamat,how='left',left_on='chemId',right_on='V1')
    original = original.drop(['chemId','V1'],1) #Remove Id
    #original = original.drop(['chemId'],1) #Remove Id    
    return original

train = merge_feamat(train,feamat)
test = merge_feamat(test,feamat)

In [4]:
#remove columns have 1 uniquene value
#print(train.nunique())

for col in train.columns:
    if (len((train[col].unique()))==1):
        train.drop(col,inplace=True,axis=1)
        test.drop(col,inplace=True,axis=1)


In [ ]:
#Try finding correlation using Pearson 
import seaborn as sb

corr = train.corrwith(train.Expected).abs()
#sb.heatmap(corr.to_frame())
train.drop(corr[corr < 0.07].index,inplace=True,axis=1)
test.drop(corr[corr < 0.07].index,inplace=True,axis=1)

    

In [5]:
#x.index[np.isinf(x).any(1)]
#test.index[np.isinf(test).any(1)]
#Drop because contain inf value
#print(train.columns.to_series()[np.isinf(train).any()])
#print(test.columns.to_series()[np.isinf(test).any()])
#train.drop('V15',1,inplace=True)
#test.drop('V15',1,inplace=True)
train.replace(np.inf,0,inplace=True)
test.replace(np.inf,0,inplace=True)


In [6]:
#split data
y = train.Expected
x = train.drop('Expected',1)

In [7]:
#normalize
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

x_scaled = pd.DataFrame(scaler.fit_transform(x),index = x.index,columns = x.columns)
test_scaled = pd.DataFrame(scaler.transform(test),index=test.index,columns = test.columns)

In [91]:
#Select K best - using ANOVA or Chi^2
from sklearn.feature_selection import SelectKBest,chi2,f_classif

selectedFeatures = SelectKBest(score_func=f_classif,k=50)
sel = selectedFeatures.fit(x_scaled,y)

dfscores = pd.DataFrame(sel.scores_)
dfcolumns = pd.DataFrame(x.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
#print(featureScores.nlargest(50,'Score'))

In [92]:
x_fs=sel.transform(x_scaled)
test_fs=sel.transform(test_scaled)

In [93]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_fs,y,test_size=0.20,random_state=42)

In [94]:
# instantiating the random over sampler 

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy=0.5,random_state=0)
# resampling X, y
x_train, y_train = ros.fit_resample(x_train, y_train)
# new class distribution 
print(Counter(y_train))

Counter({2: 53158, 1: 26579})


In [ ]:
# instantiating the random under sampler 

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
# resampling X, y
x_train, y_train = rus.fit_resample(x_train, y_train)
# new class distribution 
print(Counter(y_train))

In [ ]:
# tried over sampler with SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE()
# resampling X, y
x_train, y_train = smote.fit_resample(x_train, y_train)
# new class distribution 
print(Counter(y_train))

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())
#try to send all predicted as toxic
#df = pd.DataFrame({'Id':test_id,'Predicted':2})
#df.to_csv('result/all_2_submission.csv',index=False)

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras import regularizers
from keras import metrics as kmetrics

x_partial_train,x_validation,y_partial_train,y_validation= train_test_split(x_train,y_train,test_size=0.4,random_state=42)

model=models.Sequential()
model.add(layers.Dense(20,activation='relu',input_shape=(20,)))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_partial_train,y_partial_train,epochs=20,batch_size=5,validation_data=(x_validation,y_validation))
print("score on test: " + str(model.evaluate(x_test,y_test)[1]))
print("score on train: "+ str(model.evaluate(x_train,y_train)[1]))
print(metrics.classification_report(model.predict(x_test),y_test)) #85%
print(metrics.classification_report(model.predict(x_train),y_train)) #85%

In [13]:
def getSubmission(model,name):
    result = model.predict(test_fs)
    df = pd.DataFrame({'Id':test_id,'Predicted':result})
    #df.Predicted +=1
    df.to_csv('result/'+name+'_submission.csv',index=False)

In [14]:
from sklearn import metrics

In [30]:
#Try with random forrest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
#rf = RandomForestClassifier(random_state=42, class_weight="balanced", n_estimators=200,oob_score=True,criterion='gini',max_depth=10)
rf = RandomForestClassifier(random_state=42,n_estimators=80,oob_score=True,max_depth=30,class_weight="balanced")

rf.fit(x_train,y_train)
print(metrics.classification_report(rf.predict(x_test),y_test)) #85%
print(metrics.classification_report(rf.predict(x_train),y_train)) #85%


              precision    recall  f1-score   support

           1       0.43      0.42      0.43      2188
           2       0.91      0.91      0.91     13295

    accuracy                           0.84     15483
   macro avg       0.67      0.67      0.67     15483
weighted avg       0.84      0.84      0.84     15483

              precision    recall  f1-score   support

           1       1.00      0.99      1.00     26744
           2       1.00      1.00      1.00     52993

    accuracy                           1.00     79737
   macro avg       1.00      1.00      1.00     79737
weighted avg       1.00      1.00      1.00     79737



In [ ]:
getSubmission(rf,'randomforest_classweight_balanced') #49%

In [ ]:
#Try with decision tree
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=30)
dtc.fit(x_train,y_train)
print(metrics.classification_report(dtc.predict(x_test),y_test)) #85%
print(metrics.classification_report(dtc.predict(x_train),y_train)) #85%

In [ ]:
getSubmission(dtc,'decision_tree') #49%

In [29]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
ada = AdaBoostClassifier(n_estimators=500, random_state=42,algorithm='SAMME')
ada.fit(x_train, y_train)

print(metrics.classification_report(ada.predict(x_test),y_test)) #55%
print(metrics.classification_report(ada.predict(x_train),y_train)) #56%

              precision    recall  f1-score   support

           1       0.45      0.40      0.43      2420
           2       0.89      0.91      0.90     13063

    accuracy                           0.83     15483
   macro avg       0.67      0.66      0.66     15483
weighted avg       0.82      0.83      0.83     15483

              precision    recall  f1-score   support

           1       0.45      0.68      0.55     17706
           2       0.89      0.77      0.83     62031

    accuracy                           0.75     79737
   macro avg       0.67      0.72      0.69     79737
weighted avg       0.80      0.75      0.76     79737



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=400, learning_rate=0.01,
max_depth=12, random_state=42).fit(x_train, y_train)

print(metrics.classification_report(gbc.predict(x_test),y_test)) #78%
print(metrics.classification_report(gbc.predict(x_train),y_train)) #97%

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators':[200,300],
'learning_rate':[0.08,0.1,0.12],
'max_depth':[8,9,10]}

rds_gbc= RandomizedSearchCV(estimator=GradientBoostingClassifier(),param_distributions=params,n_iter=20,scoring='f1_macro',n_jobs=3,cv=StratifiedKFold(4),verbose=2, random_state=42)
rds_gbc.fit(x_train,y_train)
print(metrics.classification_report(rds_gbc.predict(x_test),y_test)) #77%
print(metrics.classification_report(rds_gbc.predict(x_train),y_train)) #94%


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators':[200,300],
'learning_rate':[0.08,0.1,0.12],
'max_depth':[8,9,10]}

grd_gbc= GridSearchCV(estimator=GradientBoostingClassifier(),param_grid=params,scoring='f1_macro',n_jobs=3,cv=StratifiedKFold(5),verbose=3)
grd_gbc.fit(x_train,y_train)
print(metrics.classification_report(grd_gbc.predict(x_test),y_test)) #79%
print(metrics.classification_report(grd_gbc.predict(x_train),y_train)) #95%

In [ ]:
print(rds_gbc.best_params_)

In [ ]:
getSubmission(gbc,'gradientboosting_79') #49%

In [ ]:
#preprocessing for xgb
y_train -= 1
y_test -= 1

In [95]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(n_estimators=10000,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    tree_method='gpu_hist',
    eval_metric=['error'],
    num_parallel_tree=4,
    max_depth=14,objective='binary:logistic',random_state=42)
xgbc.fit(x_train,y_train,
            verbose=True,
            early_stopping_rounds=50,
            eval_metric=['error'],
            eval_set=[(x_test,y_test)])
print(metrics.classification_report(xgbc.predict(x_test),y_test,digits=5)) #80.29%
print(metrics.classification_report(xgbc.predict(x_train),y_train,digits=5)) #95%

C:\Python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[0]	validation_0-error:0.12614
[1]	validation_0-error:0.12129
[2]	validation_0-error:0.11974
[3]	validation_0-error:0.11794
[4]	validation_0-error:0.11832
[5]	validation_0-error:0.11878
[6]	validation_0-error:0.11761
[7]	validation_0-error:0.11587
[8]	validation_0-error:0.11471
[9]	validation_0-error:0.11458
[10]	validation_0-error:0.11503
[11]	validation_0-error:0.11380
[12]	validation_0-error:0.11342
[13]	validation_0-error:0.11264
[14]	validation_0-error:0.11180
[15]	validation_0-error:0.11102
[16]	validation_0-error:0.11077
[17]	validation_0-error:0.

In [15]:
print(metrics.classification_report(xgbc.predict(x_test),y_test,digits=5)) #85%
print(metrics.classification_report(xgbc.predict(x_train),y_train,digits=5)) #85%

              precision    recall  f1-score   support

           1    0.66915   0.65481   0.66190      2193
           2    0.94324   0.94658   0.94491     13290

    accuracy                        0.90525     15483
   macro avg    0.80620   0.80069   0.80340     15483
weighted avg    0.90442   0.90525   0.90482     15483

              precision    recall  f1-score   support

           1    0.98849   0.96913   0.97871     27110
           2    0.98425   0.99419   0.98920     52627

    accuracy                        0.98567     79737
   macro avg    0.98637   0.98166   0.98395     79737
weighted avg    0.98569   0.98567   0.98563     79737



In [78]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators':[1000],
    'learning_rate':[0.1],
    'subsample':[0.7,0.8,0.9,1.0],
    'colsample_bytree':[0.7,0.8,0.9,1.0]}
fit_params={"early_stopping_rounds":50, 
            "eval_metric" : "logloss", 
            "eval_set" : [(x_test,y_test)]}


#grd_xgbc= GridSearchCV(estimator=xgbc,param_grid=params,scoring='f1_macro',n_jobs=3,verbose=3)
#grd_xgbc.fit(x_train,y_train)

rs_xgbc= RandomizedSearchCV(xgbc,params,n_iter=15,cv=5,scoring='f1_macro',n_jobs=4,verbose=3)
rs_xgbc.fit(x_train,y_train,**fit_params)



Fitting 5 folds for each of 15 candidates, totalling 75 fits
C:\Python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[0]	validation_0-logloss:0.64087
[1]	validation_0-logloss:0.59295
[2]	validation_0-logloss:0.55230
[3]	validation_0-logloss:0.51800
[4]	validation_0-logloss:0.48896
[5]	validation_0-logloss:0.46509
[6]	validation_0-logloss:0.44436
[7]	validation_0-logloss:0.42481
[8]	validation_0-logloss:0.40699
[9]	validation_0-logloss:0.39132
[10]	validation_0-logloss:0.37862
[11]	validation_0-logloss:0.36651
[12]	validation_0-logloss:0.35629
[13]	validation_0-logloss:0.34700
[14]	validation_0-logloss:0.33787

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=0.7,
                                           eval_metric=['logloss'], gamma=0,
                                           gpu_id=0, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=14, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=10000, n_jobs=4,
                                           num_parallel_tree=4, random_state=42,
                                           r

In [82]:
rs_xgbc.cv_results_

{'mean_fit_time': array([508.87295661, 464.72316742, 475.23826604, 483.06683941,
        471.42874198, 473.45618711, 454.74172025, 496.63335819,
        505.33362675, 527.67019777, 546.0872654 , 491.85641351,
        566.5989934 , 492.03718395, 439.05977249]),
 'std_fit_time': array([26.73517887, 12.74009776, 10.63233834, 23.22518893, 20.29651547,
        14.27195381, 21.88294099, 23.53010509, 18.73278156, 12.27691597,
        15.03292783, 15.00469131, 49.98719241, 18.36279898, 56.37145558]),
 'mean_score_time': array([0.93849182, 0.76633501, 0.74452524, 0.75932808, 0.66381869,
        0.73166533, 0.8158524 , 0.72952156, 0.69535804, 0.68941469,
        1.10758719, 1.00933352, 1.36895676, 1.16667795, 1.05947533]),
 'std_score_time': array([0.14050471, 0.0598361 , 0.06243187, 0.06459185, 0.05396262,
        0.05518967, 0.03753616, 0.0648947 , 0.05559987, 0.03495865,
        0.53571555, 0.56037899, 0.75454321, 0.92006421, 0.93632779]),
 'param_subsample': masked_array(data=[0.8, 0.8, 1.0,

In [79]:
pd.concat([pd.DataFrame(grd_xgbc.cv_results_["params"]),pd.DataFrame(grd_xgbc.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)
pd.concat([pd.DataFrame(rs_xgbc.cv_results_["params"]),pd.DataFrame(rs_xgbc.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)


,subsample,n_estimators,learning_rate,colsample_bytree,Accuracy
0,0.8,1000,0.1,0.8,0.928427
1,0.8,1000,0.1,0.7,0.926057
2,1.0,1000,0.1,0.7,0.933248
3,0.7,1000,0.1,0.9,0.924727
4,0.8,1000,0.1,0.9,0.928059
5,0.9,1000,0.1,0.7,0.931539
6,0.7,1000,0.1,0.7,0.922024
7,0.9,1000,0.1,0.9,0.932528
8,0.8,1000,0.1,1.0,0.928728
9,0.9,1000,0.1,1.0,0.933463


In [44]:
print(metrics.classification_report(grd_xgbc.predict(x_test),y_test,digits=5)) #79%
print(metrics.classification_report(grd_xgbc.predict(x_train),y_train,digits=5)) #95%

              precision    recall  f1-score   support

           1    0.66076   0.65039   0.65554      1147
           2    0.93936   0.94193   0.94064      6595

    accuracy                        0.89873      7742
   macro avg    0.80006   0.79616   0.79809      7742
weighted avg    0.89809   0.89873   0.89840      7742

              precision    recall  f1-score   support

           1    0.98844   0.96823   0.97823     30566
           2    0.98378   0.99416   0.98895     59257

    accuracy                        0.98534     89823
   macro avg    0.98611   0.98120   0.98359     89823
weighted avg    0.98537   0.98534   0.98530     89823



In [66]:
getSubmission(xgbc,'xgbc_rs_8033') #80.211%

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
knn.fit(x_train, y_train)


print(metrics.classification_report(knn.predict(x_test),y_test,digits=5)) #85%
print(metrics.classification_report(knn.predict(x_train),y_train,digits=5)) #85%

In [ ]:
from sklearn.svm import LinearSVC
svm=LinearSVC(C=0.0001)
svm.fit(x_train, y_train)

print(metrics.classification_report(svm.predict(x_test),y_test))
print(metrics.classification_report(svm.predict(x_train),y_train))

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
#rfecv = RFECV(estimator=RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=80,oob_score=True,criterion='gini',max_depth=5),step=1,n_jobs=3, cv=StratifiedKFold(10), scoring='f1_macro',verbose=10)
rfecv = RFECV(estimator=DecisionTreeClassifier(max_depth=20),step=1,n_jobs=3, cv=StratifiedKFold(5), scoring='f1_macro',verbose=10)
rfecv.fit(x_train,y_train)

In [ ]:
print(metrics.classification_report(rfecv.predict(x_test),y_test)) #85%
print(metrics.classification_report(rfecv.predict(x_train),y_train)) #85%

getSubmission(rfecv,'feature_elimination_balanced') #47%

In [ ]:
print('Optimal number of features: {}'.format(rfecv.n_features_))


In [ ]:
plt.figure(figsize=(16, 9))
plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('% Correct Classification', fontsize=14, labelpad=20)
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, color='#303F9F', linewidth=3)

plt.show()

In [ ]:
print(np.where(rfecv.support_ == False)[0])

X = x_train.drop(x.columns[np.where(rfecv.support_ == False)[0]], axis=1)


In [ ]:
dset = pd.DataFrame()
dset['attr'] = x_train.dtype.names
dset['importance'] = rfecv.estimator_.feature_importances_

dset = dset.sort_values(by='importance', ascending=False)


plt.figure(figsize=(16, 14))
plt.barh(y=dset['attr'], width=dset['importance'], color='#1976D2')
plt.title('RFECV - Feature Importances', fontsize=20, fontweight='bold', pad=20)
plt.xlabel('Importance', fontsize=14, labelpad=20)
plt.show()